In [2]:
%pip install gradio scikit-learn numpy 


Note: you may need to restart the kernel to use updated packages.


In [3]:
import gradio as gr
from normalization_engine import BaseEngine
from sklearn.ensemble import IsolationForest
import numpy as np

# Setup
engine_type = "petrol"
normalizer = BaseEngine(engine_type)
schema = normalizer.schema


In [4]:
def generate_normal_data(engine_type, n_samples=200):
    engine = BaseEngine(engine_type)
    schema = engine.schema
    data = []
    for _ in range(n_samples):
        sample = {
            sensor: np.random.uniform(min_val, max_val)
            for sensor, (min_val, max_val) in schema.items()
        }
        data.append(sample)
    return data

normal_data = generate_normal_data(engine_type)
X = np.array([[v for v in normalizer.normalize(d).values()] for d in normal_data])
model = IsolationForest(contamination=0.05).fit(X)


In [5]:
def detect(rpm, coolant_temp, throttle_pos, pressure, vibration, air_temp):
    input_data = {
        "rpm": rpm,
        "coolant_temp": coolant_temp,
        "throttle_pos": throttle_pos,
        "pressure": pressure,
        "vibration": vibration,
        "air_temp": air_temp
    }
    norm_input = list(normalizer.normalize(input_data).values())
    result = model.predict([norm_input])[0]
    return "⚠️ Anomaly Detected!" if result == -1 else "✅ Engine Running Normally"


In [6]:
inputs = [
    gr.Slider(800, 8000, step=100, label="RPM (Revolutions Per Minute)", value=3000),
    gr.Slider(70, 105, step=1, label="Coolant Temp (°C)", value=90),
    gr.Slider(0, 100, step=1, label="Throttle Position (%)", value=50),
    gr.Slider(2.0, 5.0, step=0.1, label="Pressure (bar)", value=3.0),
    gr.Slider(0.0, 1.5, step=0.05, label="Vibration (mm/s)", value=0.5),
    gr.Slider(15, 45, step=1, label="Air Temp (°C)", value=25)
]

iface = gr.Interface(
    fn=detect,
    inputs=inputs,
    outputs=gr.Textbox(label="System Verdict"),
    title="🚗 Engine Health Analyzer",
    description="Adjust sensor values to check if the engine is performing normally.",
    theme="default"
)

iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://2b761c2d2128729f32.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
